<a href="https://colab.research.google.com/github/the-bucketless/rapm/blob/main/rapm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5-on-5 Corsi-Based RAPM

The data is provided by [Harry Shomer](https://twitter.com/offsides_review).  Send him some love.  If the site is no longer up when you're looking at this, you can scrape the data with [his scraper](https://github.com/HarryShomer/Hockey-Scraper).

I've run this on every season to date (up to 2019/20 for full seasons and a partial 2020/21) and haven't noticed issues.  If you find something awry, let me know.  
  
I've made an effort to over-comment the code to try to help anyone new understand what's going on.

Feel free to use this as you see fit, but if you're doing something outside of personal use, keep it open source.

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer

Enter the season you want to use.  It needs to be the full first year followed by the full second year of the season, even if there weren't any games in the first year.  The earliest season available is 2007/08.

In [ ]:
season = "20192020" #@param {type: "string"}

In [ ]:
pbp_url = f"https://hockey-data.harryshomer.com/pbp/nhl_pbp{season}.csv.gz"
pbp = pd.read_csv(pbp_url, compression="gzip")

shifts_url = f"https://hockey-data.harryshomer.com/shifts/nhl_shifts{season}.csv.gz"
shifts = pd.read_csv(shifts_url, compression="gzip")

## Setting up the play-by-play dataframe

Events included in this RAPM are from regular season games at 5-on-5.  If you'd like to include playoff games, make sure you still remove shootouts from regular season games.  Leaving them in causes some issues.

The way things are set up, teams are allowed to have more than 5 skaters on the ice (this can be caused by teams trying to get too many men penalties or issues in how things were recorded) so long as both teams have a goalie on the ice.  It can also result in the odd powerplay event finding its way into the data if both teams have 5 or more skaters listed as being on the ice.

In [ ]:
venues = ("away", "home")
corsi_events = ("GOAL", "SHOT", "MISS", "BLOCK")

# converting column names to snake case (personal preference)
pbp.rename(columns={c: c.lower() for c in pbp.columns}, inplace=True)

# need to track who's a goalie for when we add in the shift changes
goalies = set([x for x in pd.unique(pbp[["away_goalie", "home_goalie"]].values.ravel()) if x == x])

# score state ranges from -3 to 3, away score state won't be needed until much later
pbp["home_score_state"] = (pbp.home_score - pbp.away_score).clip(lower=-3, upper=3)

# only keep columns that will be used
# you may want to include more if you're adding features
pbp = pbp[["game_id", "period", "event", "seconds_elapsed", "home_score_state",
           "home_zone", "ev_team", "away_team", "home_team"]]

# removing playoff games, shootouts, and events that won't factor in
pbp = pbp.loc[(pbp.game_id < 30000) & (pbp.period < 5) & (pbp.event.isin(["FAC", *corsi_events]))]

# convert seconds_elapsed to be relative to game instead of period
pbp.seconds_elapsed += (pbp.period - 1) * 1200

# corsi
pbp["corsi"] = (pbp.event.isin(corsi_events)).astype(int)
pbp["away_corsi"] = pbp.corsi * (pbp.away_team == pbp.ev_team)
pbp["home_corsi"] = pbp.corsi * (pbp.home_team == pbp.ev_team)

## Setting up the shift dataframe

In [ ]:
# more snake case
shifts.rename(columns={c: c.lower() for c in shifts.columns}, inplace=True)

# get rid of nonsense
shifts.dropna(inplace=True)
shifts = shifts.loc[(shifts.game_id < 30000) & (shifts.start < shifts.end)]

# convert times to be relative to game instead of period
shifts["start"] += (shifts["period"] - 1) * 1200
shifts["end"] += (shifts["period"] - 1) * 1200

# add away and home team from play-by-play
teams_by_venue = pbp.groupby(by="game_id", as_index=False)[["away_team", "home_team"]].first()
shifts = shifts.merge(teams_by_venue, how="left", on="game_id")

# separate players by venue
shifts["away_player"] = np.where(shifts.team == shifts.away_team, shifts.player, np.nan)
shifts["home_player"] = np.where(shifts.team == shifts.home_team, shifts.player, np.nan)

# create list of all players, this doesn't get used until the end
player_list = pd.unique(shifts.player)

## Ch-ch-ch-changes

If you're used to the EvolvingWild twins' scraper, they provide shift starts and ends for you as change events.  The data we're working on doesn't include them, so we have to add them in.  
  
This isn't the nicest looking way I've thought to do it, but it is the fastest I've found.  I've tried a lot of different ways thinking there has to be something better than the dreaded for loop, but even when I think I'm being clever, things still run slower.

The first step is to group the shifts by the unique start and end times, including columns for which players stepped on the ice and who stepped off for both teams.  The players for each side are thrown together as a set.  

In [ ]:
time_dfs = []
for change in ("start", "end"):
    time_dfs.append(shifts.groupby(by=["game_id", change], as_index=False)[["away_player", "home_player"]].agg(set))

    # rename time column for easier merging into pbp
    time_dfs[-1].rename(columns={change: "seconds_elapsed"}, inplace=True)

    # track type of change (start or end of shift)
    time_dfs[-1]["type"] = change

# combine start and end shifts into single dataframe and sort everything into place
changes = pd.concat(time_dfs, ignore_index=True).sort_values(by=["game_id", "seconds_elapsed", "type"])

for venue in venues:
    # remove na values from sets
    changes[f"{venue}_player"] = changes[f"{venue}_player"].apply(lambda x: {player for player in x if player == player})

    # separate changes by players going on and players going off
    changes[f"{venue}_on"] = np.where(changes["type"] == "start", changes[f"{venue}_player"], "")
    changes[f"{venue}_off"] = np.where(changes["type"] == "end", changes[f"{venue}_player"], "")

# bring players going on and players going off at the same time into single row
changes = changes.groupby(by=["game_id", "seconds_elapsed"], as_index=False).agg({"away_on": "last", "away_off": "first",
                                                                                  "home_on": "last", "home_off": "first"})

Then, we loop through every change, keeping track of who's still on the ice using set operations.  For each row, we start with who was on the ice in the previous row, then remove everyone who left the ice, and add anyone starting their shift.  It's important to remove players before adding because the end of one period shares its time with the start of the next.  Goalies will often have shifts from 0 to 1200, 1200 to 2400, etc.  If you add before you remove, the goalie won't be out there to start the 2nd.

In [ ]:
on_ice_col = {"away": [], "home": []}    # columns to be added after the for loop
on_ice = {"away": set(), "home": set()}    # which players are currently on the ice
for row in changes.itertuples():
    for venue in venues:
        # remove the players going off, then add the players going on
        on_ice[venue] = on_ice[venue].difference(getattr(row, f"{venue}_off")).union(getattr(row, f"{venue}_on"))
        on_ice_col[venue].append(on_ice[venue])

# add to dataframe
changes["away_players"] = on_ice_col["away"]
changes["home_players"] = on_ice_col["home"]

Next, we split the players out of the sets and into their own columns.  Empty net situations are noted and goalies are removed - we don't need RAPM scores for them.  Because more than 5 skaters are allowed to be on the ice, we need to keep the number of home and away columns consistent.  Missing values are replaced with "dummy" to allow a future fill function to work properly.

In [ ]:
for venue in venues:
    # empty net if no one on the ice is a goalie
    changes[f"{venue}_empty_net"] = changes[f"{venue}_players"].apply(lambda x: len(x.intersection(goalies)) == 0).astype(int)

    # remove goalies from player list
    changes[f"{venue}_players"] -= goalies

    # split sets into separate columns
    players = pd.DataFrame(changes[f"{venue}_players"].values.tolist()).add_prefix(venue)

    # glue everything together
    changes = changes.join(players)

# allowing more than 5 skaters on the ice, need to keep the number of home and away columns consistent
max_away = int(changes.columns[list(changes.columns).index("home_empty_net") - 1][-1])
max_home = int(changes.columns[-1][-1])

for i in range(max_away, max_home):
    changes[f"away{i + 1}"] = "dummy"
for i in range(max_home, max_away):
    changes[f"home{i + 1}"] = "dummy"

# add some columns for the merge
changes["event"] = "CHANGE"
changes["away_corsi"] = 0
changes["home_corsi"] = 0

# remove columns that won't survive the merge
changes.drop(columns=[f"{venue}_{c}" for venue in venues for c in ("on", "off", "players")], inplace=True)

# fill all na so they don't interfere with later fillings
changes.fillna("dummy", inplace=True)

Finally, we have to make sure things go in the right place.  Changes are considered to be [start time, end time).  That's to say, players are no longer considered to be on the ice when the last second of their shift comes along.  The one exception is if a shot both precedes and occurs at the same time as a faceoff (shot, save, whistle all in the same second).  For those situations, players will be on the ice for the shot, but not the faceoff.  This bit of nonsense accomplishes that.

In [ ]:
shifted_pbp = pbp.shift(-1)
pbp["sort_code"] = np.where((shifted_pbp.event == "FAC") & (shifted_pbp.seconds_elapsed == pbp.seconds_elapsed), 0, 2)
changes["sort_code"] = 1

## Merging

Everything's going into a single dataframe.

In [ ]:
# add all the changes to the end of the play-by-play dataframe
pbp = pd.concat([pbp, changes], ignore_index=True)

# sort them into place
pbp.sort_values(by=["game_id", "seconds_elapsed", "sort_code"], inplace=True)

Add a couple features, fill in some missing values, and remove everything we don't need.

In [ ]:
# track zone starts
# need zones for home and away teams on faceoffs
pbp["away_zone"] = np.select([pbp.home_zone == "Off", pbp.home_zone == "Def"], ["Def", "Off"], default="Neu")

# zone starts will be the zone of the faceoff until someone changes, then OTF (on-the-fly)
pbp["away_start_zone"] = np.select([pbp.event == "FAC", pbp.event == "CHANGE"], [pbp.away_zone, "OTF"], default=pd.NA)
pbp["home_start_zone"] = np.select([pbp.event == "FAC", pbp.event == "CHANGE"], [pbp.home_zone, "OTF"], default=pd.NA)

# fill all na values by pushing things down the dataframe
pbp.fillna(method="ffill", inplace=True)

# time between events, negative values occur when moving on to the next game
# these get clipped to 0 and removed with the next line
pbp["duration"] = (pbp.seconds_elapsed.shift(-1) - pbp.seconds_elapsed).clip(lower=0)

# get rid of anything that doesn't have a corsi value and is of 0 length
pbp = pbp[(pbp.home_corsi > 0) | (pbp.away_corsi > 0) | (pbp.duration > 0)]

# only using 5-on-5
pbp = pbp.loc[(pbp.away_empty_net == 0) & (pbp.home_empty_net == 0) 
              & (pbp.away4 != "dummy") & (pbp.home4 != "dummy")]

## Stints

Group everything into stints.  We have to add up the corsi values and the stint lengths based on any events that have the same features.

In [ ]:
# group by all the features we're using
stints = pbp.groupby(by=["period", "away_start_zone", "home_start_zone", "home_score_state",
                         *[c for c in pbp.columns if ("away" in c or "home" in c) and "_" not in c]],
                     as_index=False, sort=False)
stints = stints[["away_corsi", "home_corsi", "duration"]].sum()

# add the away score state for easy use in the upcoming for loop
stints["away_score_state"] = -stints.home_score_state

# some stints are of length 0 but have corsi events - bump their stint lengths up to 0.5
stints.duration = stints.duration.clip(lower=0.5)

We have to duplicate every stint.  In one case, the away team is considered to be on offense, in the other, the home team is attacking.

In [ ]:
venue_stints = []
for venue in venues:
    def_venue = "home" if venue == "away" else "away"

    venue_stints.append(stints.copy())

    # score state relative the offensive team
    venue_stints[-1]["score_state"] = venue_stints[-1][f"{venue}_score_state"]

    # corsi per 60 for the offensive team
    venue_stints[-1]["corsi60"] = venue_stints[-1][f"{venue}_corsi"] / venue_stints[-1].duration * 3600

    # remove columns that are no longer needed
    venue_stints[-1].drop(columns=["home_score_state", "away_score_state", "home_corsi", "away_corsi"], inplace=True)

    # change all columns to be off/def rather than home/away
    venue_stints[-1].columns = venue_stints[-1].columns.str.replace(venue, "off")
    venue_stints[-1].columns = venue_stints[-1].columns.str.replace(def_venue, "def")

    # add a feature for home ice
    venue_stints[-1]["is_home"] = int(venue == "home")

# combine away as offense and home as offense into single dataframe
all_stints = pd.concat(venue_stints, ignore_index=True)

# dummy categorical features (other than players)
all_stints = pd.get_dummies(data=all_stints, columns=["period", "off_start_zone", "def_start_zone", "score_state"])

Now, we dummy the players.  The below relies on the columns of all_stints having all the offensive players first, then the defensive players, then duration.  If you're adding features, make sure this ordering remains (ensure new columns appear after duration).

In [ ]:
for side in ("off", "def"):
    # these are the columns that immediately follow the offensive player and defensive player columns
    next_column = "def0" if side == "off" else "duration"

    # get all the player columns for off or def
    cols = all_stints.loc[:, :next_column].iloc[:, :-1]

    # create dummies
    mlb = MultiLabelBinarizer(sparse_output=True)
    players = mlb.fit_transform(cols.astype(str).values)
    
    # glue everything together
    all_stints = all_stints.join(
        pd.DataFrame.sparse.from_spmatrix(players, index=all_stints.index, 
                                          columns=[f"{side}_{player}" for player in mlb.classes_]))

    # get rid of the non-dummied player columns
    all_stints.drop(columns=cols.columns, inplace=True)

    # don't need the "dummy" player variable
    dummy_col = f"{side}_dummy"
    if dummy_col in all_stints.columns:
        all_stints.drop(columns=[dummy_col], inplace=True)

## RAPM

Convert all_stints into the design matrix (X), targets (y), and weights (w).

In [ ]:
y = all_stints.pop("corsi60")
w = all_stints.pop("duration")
X = csr_matrix(all_stints.astype(pd.SparseDtype("float", 0.0)).sparse.to_coo())

Because sklearn's scoring metrics don't concern themselves with the sample weights of the model, we need to create our own scorer that does.

In [ ]:
def mse(y, y_pred, sample_weight=None):
    if sample_weight is not None:
        sample_weight = sample_weight.loc[y.index.values].values.reshape(-1)
    
    return mean_squared_error(y, y_pred, sample_weight=sample_weight)

mse_scorer = make_scorer(mse, greater_is_better=False, sample_weight=w)

Time for some cross validation to find the best alpha.  Depending where you look, this parameter may be called lambda, but sklearn calls it alpha.  I've only included three values here so that everything runs fairly quickly.  There's no guarantee any of the three will be all that great, so I'd encourage you to play with this to find something better.  When I train my RAPM model, I use the hyperopt package, but I decided to keep things a little simpler here.

In [ ]:
model_cv = RidgeCV(alphas=[25000, 30000, 35000], cv=10, scoring=mse_scorer).fit(X, y, sample_weight=w)

The best alpha gets plugged into the final model.

In [ ]:
model = Ridge(alpha=model_cv.alpha_).fit(X, y, sample_weight=w)

## Scores

All that's left is to look at the scores.  We'll make a dataframe consisting of only the players' scores, though it can be interesting to look at how the other features affect things.  If you add features, make sure you give them names longer than 4 characters for this to work right.

In [ ]:
rapm = pd.DataFrame({"player": [x[4:] for x in all_stints.columns], 
                     "side": [x[:3] for x in all_stints.columns], 
                     "score": model.coef_})

# separate columns for offense and defense
rapm["off"] = rapm.score * (rapm.side == "off")
rapm["def"] = rapm.score * (rapm.side == "def")

# group each player into their own row
rapm = rapm.loc[rapm.player.isin(player_list), ["player", "off", "def"]]
rapm = rapm.groupby(by="player").sum()

# negatives values are good for defense
rapm["total"] = rapm.off - rapm["def"]

Let's sort by total RAPM score.

In [ ]:
rapm.sort_values(by="total", ascending=False)

,off,def,total
player,,,
VALERI NICHUSHKIN,4.782407,-5.148859,9.931266
TOMAS TATAR,7.554177,-1.891966,9.446143
JARED SPURGEON,5.060905,-3.493998,8.554903
CRAIG SMITH,6.056402,-2.476526,8.532928
MAX PACIORETTY,5.673644,-2.848287,8.521931
...,...,...,...
BRETT HOWDEN,-3.443737,4.997895,-8.441632
MAX COMTOIS,-5.276913,3.258515,-8.535428
NIKITA ZAITSEV,-3.386665,5.356667,-8.743331


This is if you'd like to see a specific player's score.  Note that the name has to exactly match what's in the dataframe.

In [ ]:
player_name = "SIDNEY CROSBY" #@param {type: "string"}
rapm.loc[player_name]

off      3.385302
def      2.323679
total    1.061623
Name: SIDNEY CROSBY, dtype: float64